In [14]:
import pandas as pd
import numpy as np
from basic_ollama_agent import OllamaAgent
import ollama
import json
import inspect
from typing import List, Callable, Optional, Any, Dict
from pydantic import BaseModel

In [4]:
df = pd.read_excel("bank_earnings_data_2019-01-01_2025-12-31.xlsx", sheet_name="Bank_Earnings_Data")
df.head()

,Datetime,CompanyName,TotalRevenue,InterestIncome,NonInterestIncome,InterestExpense,NetInterestIncome,ProvisionForLoanLosses,NonInterestExpense,NetIncome,EarningsPerShare,TotalAssets,TotalLoans,TotalDeposits,ShareholdersEquity
0,2019-03-31,AMERICAN EXPRESS COMPANY,6.697000e+09,2.725000e+09,8.305000e+09,8.950000e+08,2.059000e+09,5.250000e+08,7.597000e+09,1.550000e+09,1.81,1.971930e+11,NaN,7.285700e+10,22218000000
1,2019-06-30,AMERICAN EXPRESS COMPANY,1.377600e+10,5.489000e+09,1.706900e+10,1.786000e+09,4.133000e+09,1.128000e+09,1.535500e+10,3.311000e+09,3.88,1.976030e+11,NaN,7.259000e+10,23092000000
2,2019-09-30,AMERICAN EXPRESS COMPANY,2.082700e+10,8.374000e+09,2.585500e+10,2.663000e+09,6.336000e+09,1.732000e+09,2.319900e+10,5.066000e+09,5.97,1.941840e+11,NaN,7.329800e+10,23025000000
3,2019-12-31,AMERICAN EXPRESS COMPANY,2.815900e+10,1.130800e+10,3.493600e+10,3.464000e+09,8.620000e+09,2.462000e+09,3.155400e+10,6.759000e+09,8.00,1.983210e+11,NaN,7.328700e+10,23071000000
4,2020-03-31,AMERICAN EXPRESS COMPANY,6.296000e+09,2.909000e+09,7.980000e+09,7.160000e+08,2.330000e+09,1.876000e+09,7.237000e+09,3.670000e+08,0.41,1.860600e+11,NaN,7.796200e+10,21006000000


In [12]:
### Generate prompt
role = "You are an expert Earnings Data Extractor and Analyzer. " 
task = "Call the appropriate functions to extract the earnings data from the DataFrame and analyze it\n"

Context = "Here are the metrics present in the data, enclosed in <column> tags: \n<columns>" + f"{', '.join(df.columns.tolist()[2:])}" + "</columns>"
prompt = role + task + Context

print(prompt)


You are an expert Earnings Data Extractor and Analyzer. Call the appropriate functions to extract the earnings data from the DataFrame and analyze it
Here are the metrics present in the data, enclosed in <column> tags: 
<columns>TotalRevenue, InterestIncome, NonInterestIncome, InterestExpense, NetInterestIncome, ProvisionForLoanLosses, NonInterestExpense, NetIncome, EarningsPerShare, TotalAssets, TotalLoans, TotalDeposits, ShareholdersEquity</columns>


## Basic Agent

In [ ]:
# Example tool functions
def get_product(a: int, b: int) -> int:
    """
    Computes the product of two numbers.
    """
    return int(a) * int(b)

def calculate_sum(a: int, b: int) -> int:
    """
    Calculate the sum of two numbers.
    
    Args:
        a: First number
        b: Second number
    """
    return a + b

# Example output schema
class WeatherResponse(BaseModel):
    location: str
    temperature: float
    units: str
    description: str

# Create agent
agent = OllamaAgent(
    model_name="llama3.2:1b",
    tools=[get_product, calculate_sum],
    output_schema=None
)

# Use agent
result = agent.invoke("What's the Sum of 11 and 22? Also, what's the product of 11 and 22?")
print(result)